<br>

**This jupyter notebook touches on**
* **GPU**
* <u>**Tensor**</u>
* <u>**Hidden Layers**</u>
* **Nodes**

<br>

## **Step 1: Import**

In [1]:
import pandas as pd
from datetime import datetime
import random
from random import randint
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

<br>

## **Step 2: Make**

In [2]:
# for reproducibility
random.seed(45)
torch.manual_seed(0)

dataset = pd.DataFrame(index=range(100_000), columns=['X1','X2','error','y'])
dataset['X1'] = dataset.apply(lambda _: randint(5,15), axis=1)
dataset['X2'] = dataset.apply(lambda _: randint(5,15), axis=1)
dataset['error'] = dataset.apply(lambda _: randint(-2,2), axis=1)
dataset['y'] = (dataset['X1']**2 + dataset['X2'])/(dataset['X1'] + dataset['X2']) + dataset['error']

In [3]:
dataset.head()

,X1,X2,error,y
0,9,6,1,6.800000
1,11,13,0,5.583333
2,12,10,1,8.000000
3,9,5,1,7.142857
4,6,5,1,4.727273


In [4]:
dataset = dataset[['X1','X2','y']]

In [5]:
dataset.head()

,X1,X2,y
0,9,6,6.800000
1,11,13,5.583333
2,12,10,8.000000
3,9,5,7.142857
4,6,5,4.727273


<br>

## **Step 3: Assign features (X) and target (y)**

In [6]:
X = dataset.iloc[:, 0:2]
y = dataset.iloc[:, 2]

In [7]:
X[:5]

,X1,X2
0,9,6
1,11,13
2,12,10
3,9,5
4,6,5


In [8]:
y[:5]

0    6.800000
1    5.583333
2    8.000000
3    7.142857
4    4.727273
Name: y, dtype: float64

<br>

## **Step 4: Convert from Pandas dataframe to Numpy array**

In [9]:
X = X.to_numpy()
y = y.to_numpy()

In [10]:
X[:5]

array([[ 9,  6],
       [11, 13],
       [12, 10],
       [ 9,  5],
       [ 6,  5]], dtype=int64)

In [11]:
y[:5]

array([6.8       , 5.58333333, 8.        , 7.14285714, 4.72727273])

<br>

## **Step 5: Convert from Numpy array to PyTorch tensor**

In [12]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [13]:
X[:5]

tensor([[ 9.,  6.],
        [11., 13.],
        [12., 10.],
        [ 9.,  5.],
        [ 6.,  5.]])

In [14]:
y[:5]

tensor([[6.8000],
        [5.5833],
        [8.0000],
        [7.1429],
        [4.7273]])

<br>

## **Step 6: Set up GPU**

In [15]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


<br>

## **Step 7: Push tensor to GPU**

In [16]:
X = X.to(device)
y = y.to(device)

In [17]:
X[:5]

tensor([[ 9.,  6.],
        [11., 13.],
        [12., 10.],
        [ 9.,  5.],
        [ 6.,  5.]], device='cuda:0')

In [18]:
y[:5]

tensor([[6.8000],
        [5.5833],
        [8.0000],
        [7.1429],
        [4.7273]], device='cuda:0')

<br>

## **Step 8: Set up custom dataset**

In [19]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.y = y
        self.X = X
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        y = self.y[idx]
        X = self.X[idx]
        return X, y

In [20]:
df_dataset = CustomDataset(X, y)

In [21]:
k = 0
for i, j in df_dataset:
    print(i, '\t', j)
    k = k + 1
    if k == 5:
        break

tensor([9., 6.], device='cuda:0') 	 tensor([6.8000], device='cuda:0')
tensor([11., 13.], device='cuda:0') 	 tensor([5.5833], device='cuda:0')
tensor([12., 10.], device='cuda:0') 	 tensor([8.], device='cuda:0')
tensor([9., 5.], device='cuda:0') 	 tensor([7.1429], device='cuda:0')
tensor([6., 5.], device='cuda:0') 	 tensor([4.7273], device='cuda:0')


<br>

## **Step 9: Set up custom dataloader with batch of 128**

In [22]:
df_dataloader = DataLoader(df_dataset, batch_size=128, shuffle=False)

In [23]:
k = 0
for i, j in df_dataloader:
    print(pd.DataFrame(zip(i, j), columns=['X','y']))
    k = k + 1
    if k == 2:   # print out 2 batches
        break

                                                     X  \
0    [tensor(9., device='cuda:0'), tensor(6., devic...   
1    [tensor(11., device='cuda:0'), tensor(13., dev...   
2    [tensor(12., device='cuda:0'), tensor(10., dev...   
3    [tensor(9., device='cuda:0'), tensor(5., devic...   
4    [tensor(6., device='cuda:0'), tensor(5., devic...   
..                                                 ...   
123  [tensor(6., device='cuda:0'), tensor(5., devic...   
124  [tensor(6., device='cuda:0'), tensor(14., devi...   
125  [tensor(6., device='cuda:0'), tensor(13., devi...   
126  [tensor(7., device='cuda:0'), tensor(8., devic...   
127  [tensor(11., device='cuda:0'), tensor(9., devi...   

                                     y  
0    [tensor(6.8000, device='cuda:0')]  
1    [tensor(5.5833, device='cuda:0')]  
2        [tensor(8., device='cuda:0')]  
3    [tensor(7.1429, device='cuda:0')]  
4    [tensor(4.7273, device='cuda:0')]  
..                                 ...  
123  [tensor(2.7

<br>

## **Step 10: Set up model and push model to GPU**

In [24]:
# Hidden layer
#    Multi-Layer Perceptron = ReLU
#    Convolutional Neural Network = ReLU
#    Recurrent Neural Network = Sigmoid or Tanh

# Output layer
#    Regression = Linear
#    Binary = Sigmoid
#    Multi-class = Softmax
#    Multi-label = Sigmoid

model = nn.Sequential(
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.ReLU(),
    nn.Linear(2, 1)
).to(device)
print(model)

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=2, bias=True)
  (3): ReLU()
  (4): Linear(in_features=2, out_features=2, bias=True)
  (5): ReLU()
  (6): Linear(in_features=2, out_features=2, bias=True)
  (7): ReLU()
  (8): Linear(in_features=2, out_features=2, bias=True)
  (9): ReLU()
  (10): Linear(in_features=2, out_features=2, bias=True)
  (11): ReLU()
  (12): Linear(in_features=2, out_features=1, bias=True)
)


In [25]:
next(model.parameters()).is_cuda

True

<br>

## **Step 11: Train model**

In [26]:
loss_fn = nn.MSELoss()   # Mean Square Error
optimizer = optim.Adam(model.parameters(), lr=0.001)   # Learning rate = 0.001

n_epochs = 10

time_start = datetime.now()

for epoch in range(n_epochs):
    for Xbatch, ybatch in df_dataloader:
        optimizer.zero_grad()
        y_pred = model(Xbatch)
        loss = loss_fn(y_pred, ybatch)
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

time_end = datetime.now()
print()
print('Time difference in seconds :', (time_end-time_start).total_seconds())
print('Time difference in minutes :', (time_end-time_start).total_seconds()/60)



Finished epoch 0, latest loss 14.3751220703125
Finished epoch 1, latest loss 7.514801025390625
Finished epoch 2, latest loss 7.629067420959473
Finished epoch 3, latest loss 7.6519269943237305
Finished epoch 4, latest loss 7.649843215942383
Finished epoch 5, latest loss 7.647741317749023
Finished epoch 6, latest loss 7.645987510681152
Finished epoch 7, latest loss 7.6447553634643555
Finished epoch 8, latest loss 7.644020080566406
Finished epoch 9, latest loss 7.6436309814453125

Time difference in seconds : 20.426454
Time difference in minutes : 0.3404409


<br>

## **Step 12: Use trained model to compute accuracy on current data**
**Note that the actual data science workflow should split data into training and testing**

In [27]:
y_pred = model(X)

In [28]:
y_pred[:5]

tensor([[5.7434],
        [5.7434],
        [5.7434],
        [5.7434],
        [5.7434]], device='cuda:0', grad_fn=<SliceBackward0>)

In [29]:
dataset['y_pred'] = y_pred.cpu().detach().numpy()

In [30]:
dataset.head()

,X1,X2,y,y_pred
0,9,6,6.800000,5.743443
1,11,13,5.583333,5.743443
2,12,10,8.000000,5.743443
3,9,5,7.142857,5.743443
4,6,5,4.727273,5.743443
